In [ ]:
'''
Investing in stocks that are cheapest relative to common measures of business value
Selects 50 best stocks from list of stocks
'''

In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats

In [ ]:
stocks = pd.read_csv('stocks.csv')
from apitoken import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'aapl'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

In [ ]:
price = data['peRatio']
price

In [ ]:
#batch api call
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Ticker', 'Price', 'P-E']

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                ],
                index = my_columns),
            ignore_index = True)
        
final_dataframe

In [ ]:
#choose top value stocks
final_dataframe.sort_values('P-E', inplace = True)
final_dataframe = final_dataframe[final_dataframe['P-E'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)

In [ ]:
final_dataframe

In [ ]:
#calculating the number of shares to buy
while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio in £: '))
        if portfolio_size > 0:
            break
        print('Please Enter a valid number: ')
    except ValueError:
        print('Please Enter a valid number: ')

In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = round(position_size/final_dataframe.loc[row, 'Price'], 5)
    
final_dataframe

In [ ]:
#more realistic value strategy
'''
using price to earning ratio
price to book ratio
price to sales ratio
enterprise value divided by earnings before interest, taxes, depreciation, and amortization
enterprise value divided by gross profit
'''

In [ ]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
data

pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

ev = data[symbol]['advanced-stats']['enterpriseValue']

ev_to_ebitda = ev/data[symbol]['advanced-stats']['EBITDA']
ev_to_gross = ev/data[symbol]['advanced-stats']['grossProfit']

In [ ]:
rv_columns = ['Ticker',
             'Price',
             'Number of Shares to Buy',
             'P-E Ratio',
             'P-E Percentile',
             'P-B Ratio',
             'P-B Percentile',
             'P-S Ratio',
             'PS Percentile',
             'EV/EBITDA',
             'EV/EBITDA Percentile',
             'EV/GP',
             'EV/GP Percentile',
             'RV Score']

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = ev/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross = ev/gp
        except TypeError:
            ev_to_gross = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross,
            'N/A',
            'N/A'
            ],
            index = rv_columns),
            ignore_index = True)

In [ ]:
rv_dataframe

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

In [ ]:
for column in ['P-E Ratio', 'P-B Ratio', 'P-S Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [ ]:
rv_dataframe

In [ ]:
metrics = {'P-E Ratio':
    'P-E Percentile',
    'P-B Ratio':
    'P-B Percentile',
    'P-S Ratio':
    'PS Percentile',
    'EV/EBITDA':
    'EV/EBITDA Percentile',
    'EV/GP':
    'EV/GP Percentile'}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

In [ ]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

In [ ]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe.reset_index(inplace = True, drop = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe

In [ ]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for i in rv_dataframe.index:
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = round(position_size/rv_dataframe.loc[i, 'Price'], 5)

rv_dataframe